## Estid Lozano
## David Herrera

In [ ]:
# imports
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Exercise 1

**1.1.** Write a function **decode_image_dataset(X, size, channels)** that takes a numpy array **X** with *n* rows and returns a numpy array of shape *n x size x size x channels*.
Assume that within **X** the first *size x size* attributes are for the first channel, the second *size x size* attributes for the second channel, etc.

In [ ]:
def decode_image_dataset(X, size, channels):
    X_decoded = X.reshape((len(X), size, size, channels), order='F')
    return np.transpose(X_decoded, (0, 2, 1, 3)) # Rotates the img

**1.2.** Load the **imgds.csv** dataset and visualize some of the images together with their label (check the imshow function of matplotlib).

In [ ]:
X = pd.read_csv("imgds.csv").to_numpy()
y, X = X[:, -1], X[:, :-1]
img_size = int(math.sqrt(X.shape[1] / 3))
y_onehot = to_categorical(y)
X_decoded = decode_image_dataset(X, img_size, 3)

In [ ]:
rows, cols, figsize, = 2, 5, 12
plt.figure(figsize=(figsize, figsize * rows / cols))
for i, x in enumerate(X_decoded[:rows * cols]):
    plt.subplot(rows, cols, i + 1)
    plt.title("label: " + str(y[i]))
    plt.imshow(x)
plt.show()

# Exercise 2

We now work with the imgds dataset, which is a ten class classification problem.

**2.1.** Create fully connected networks in Keras, and train them on the original unformatted data. Try 1 to 10 hidden layers with between 10, 20, 30, ..., 100 neurons, i.e. 100 architectures in total. Use a softmax function in the last layer.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2)

# setup and save models in tuples (model, num_layers, units_per_layer)
layerss = range(1, 11)
units = range(10, 110, 10)
modelsList = [(
    keras.Sequential(
        [layers.Dense(n_unit, activation='relu', input_dim=X.shape[1])]
        + [layers.Dense(n_unit, activation='relu') for i in range(n_layer - 1)] +
        [layers.Dense(10, activation='softmax')]
    ),
    n_layer, n_unit
) for n_layer in layerss for n_unit in units]

for model in modelsList:
    model[0].compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Create a matrix with performances and show them in a heat map (imshow).

In [ ]:
acc = np.zeros((len(layerss),len(units)))
for model in modelsList:
    acc[model[1]-1][int((model[2]/10)-1)] = model[0].fit(X_train, y_train, epochs=10, batch_size=50, validation_data=(X_test, y_test)).history['accuracy'][-1]
fig, ax = plt.subplots()
im = ax.imshow(acc)
cbar = ax.figure.colorbar(im, ax=ax)
cbar.ax.set_ylabel("accuracy", rotation=-90, va="bottom")
ax.set_xticks(np.arange(acc.shape[1]))
ax.set_yticks(np.arange(acc.shape[0]))
ax.set_xlabel("layers")
ax.set_xticklabels(layerss)
ax.set_ylabel("units")
ax.set_yticklabels(units)
for i in range(acc.shape[0]):
    for j in range(acc.shape[1]):
        text = ax.text(j, i, acc[i, j],ha="center", va="center", color="w")

**2.2.** Create convolutional networks in Keras, and train them on the formatted data. Use at least one dense layer prior to the output layer. Check out the **Flatten** layer to convert the output of a convolutional layer into a vector-based representation necessary for the dense layer.

Try the combinations of the following configurations:
<div style="margin-left: 1em;">
a) numbers of 2D convolutional layers between 1 and 5 (we do not convolve in 3D here, but apply 2D convolutions over each of the color channels). <br/>
b) numbers of filters between 1 and 100. <br/>
c) window sizes (kernel_size) between 2 and 10. <br/>
d) no pooling, avg pooling and max pooling.
</div>

In [ ]:
del X, y,img_size,modelsList
X_train, X_test, y_train, y_test = train_test_split(X_decoded, y_onehot, test_size=0.2)
del y_onehot, X_decoded

Report your results graphically and draw a conclusion.

In [ ]:
#create models

#variables
convulationalLayers = list(range(1,5))
filters = [int(round((100**(1/9.0))**i)) for i in range(10)]
kernelSizes = list(range(2,10))
poolings= [None,layers.MaxPooling2D,layers.AveragePooling2D]


modelsList = []
for conLayer in convulationalLayers:
    for fil in filters:
        for kerSize in kernelSizes:
            for pool in poolings:
                model = models.Sequential()
                n = 32
                for i in range(conLayer):
                    if i == 0:
                        model.add(layers.Conv2D(fil, (kerSize, kerSize), activation='relu', input_shape=(32, 32, 3)))
                        n = n - kerSize +1
                        if pool:
                            model.add(pool((kerSize, kerSize)))
                            n = n /kerSize
                    else:
                        if n - kerSize +1>1:
                            model.add(layers.Conv2D(fil, (kerSize, kerSize), activation='relu'))
                            n = n - kerSize +1
                            if pool:
                                if n /kerSize>1:
                                    model.add(pool((kerSize, kerSize)))
                                    n =n /kerSize
                model.add(layers.Flatten())
                model.add(layers.Dense(10, activation='softmax'))
                model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
                modelsList.append({'model':model,'title': 'ConvLayers:'+str(conLayer)+', filters:'+ str(fil)+',kernelSize:'+ str(kerSize)+",pooltype:"+ str(pool) })
                


In [ ]:
maxAccuracy = [0,'None']
plt.figure(figsize=(12, 10), dpi=80)
for model in modelsList:
    print('training:'+model["title"])
    history =  model['model'].fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
    plt.plot(history.history['accuracy'], label=model['title'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0, 1])
    plt.legend(loc='lower right')
    if history.history['accuracy'][-1]>maxAccuracy[0]:
        maxAccuracy[0] = history.history['accuracy'][-1]
        maxAccuracy[1] = model['title']
print('bestModel',maxAccuracy)

In [ ]:
# best structure found

model = models.Sequential()
# 160 = 32 * 5
model.add(layers.Conv2D(160, (2, 2), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(160, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(160, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history =  model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

plt.plot(history.history['accuracy'], label='accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

Conclusion: Through different combinations we found that the best kernelSize was 2, on the other hand 2 and 3 layers present a very good result, with a MaxPooloing, with 96,128 or 160 filters

These were the results found:

* ConvLayers:2, filters:96,kernelSize:2,MaxPooling2D
accuracy:0.645312488079071
* ConvLayers:2, filters:128,kernelSize:2,MaxPooling2D
accuracy:0.6386874914169312
* ConvLayers:2, filters:160,kernelSize:2,MaxPooling2D
accuracy:0.6569374799728394

* ConvLayers:3, filters:96,kernelSize:2,MaxPooling2D
accuracy:0.6629375219345093
* ConvLayers:3, filters:128,kernelSize:2,MaxPooling2D
accuracy:0.692354142665863
* ConvLayers:3, filters:160,kernelSize:2,MaxPooling2D
accuracy:0.7023749947547913

